# mnist_nn 구조 확인

In [1]:
import torch
import torch.nn as nn

In [4]:
inputs = torch.Tensor(1, 1, 28, 28)
inputs

tensor([[[[ 0.0000e+00,  0.0000e+00,  3.4752e-43,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  1.9285e-14,  4.5775e-41,  2.3237e-24,  4.5775e-41,
            0.0000e+00,  0.0000e+00,         nan,         nan,  0.0000e+00,
            0.0000e+00,  1.4013e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            1.2612e-44,  0.0000e+00,  3.1445e-13,  4.5775e-41,  7.5546e-37,
            4.5775e-41,         nan,         nan],
          [ 0.0000e+00,  0.0000e+00,         nan,         nan,  0.0000e+00,
            0.0000e+00,  1.4013e-45,  0.0000e+00,  1.0089e-43,  0.0000e+00,
            1.5414e-44,  0.0000e+00,  3.1445e-13,  4.5775e-41,  7.5564e-37,
            4.5775e-41,  8.4078e-45,  0.0000e+00,  1.4013e-44,  0.0000e+00,
                   nan,         nan,  0.0000e+00,  0.0000e+00,  1.4013e-45,
            4.5775e-41,  2.0179e-43,  0.0000e+00],
          [ 1.5414e-44,  0.0000e+00,  3.1445e-13,  4.5775e-41,  7.5568e-37,
            4.5775e-41,         nan,         nan,  3.6434e-44,

In [5]:
inputs.shape

torch.Size([1, 1, 28, 28])

In [16]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
pool = nn.MaxPool2d(2)
conv2 = nn.Conv2d(32, 64, 3, padding=1)

out = conv1(inputs)
print(out.shape)
out = pool(out)
print(out.shape)
out = conv2(out)
print(out.shape)
out = pool(out)
print(out.shape)

out = out.view(out.size(0), -1)
print(out.shape)

fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])
torch.Size([1, 3136])
torch.Size([1, 10])


# mnist_nn 구현

In [18]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    
learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [24]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out      

In [25]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
# training
total_batch = len(data_loader)
print("Learning Started")

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader: # X는 이미지, Y는 레이블
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))

print('Learning Finished!')

Learning Started
[Epoch:1] cost = 0.22401826083660126
[Epoch:2] cost = 0.061247751116752625
[Epoch:3] cost = 0.046651650220155716
[Epoch:4] cost = 0.03635840117931366
[Epoch:5] cost = 0.0316811203956604
[Epoch:6] cost = 0.025749454274773598
[Epoch:7] cost = 0.02198890596628189
[Epoch:8] cost = 0.01837753877043724
[Epoch:9] cost = 0.015107797458767891
[Epoch:10] cost = 0.014147985726594925
[Epoch:11] cost = 0.011075343005359173
[Epoch:12] cost = 0.009764043614268303
[Epoch:13] cost = 0.008704242296516895
[Epoch:14] cost = 0.00746592553332448
[Epoch:15] cost = 0.00554725481197238
Learning Finished!


In [27]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/Users/gyuyeonjo/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/gyuyeonjo/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9854000210762024
